In [1]:
# PRE-STEP: Install Required Dependencies
%pip install langchain
%pip install langgraph
%pip install langchain-openai
%pip install chromadb
%pip install python-dotenv
%pip install pydantic


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 

In [ ]:
# Cell 1: Establish baseline agent with episodic and semantic memory
# NOTE: We are creating a baseline memory store that will only be used for demonstration purposes in steps 1-5
import os
import sys
from typing import Dict, List, Optional
from pydantic import BaseModel, Field
from baseline_agent import CoALABaselineAgent
from domain_investment.investment_advisor_agent import InvestmentAdvisorAgent
from procedural_memory import ProceduralMemory
from domain_investment.investor_test_scenarios import (
    setup_hierarchy_demo, get_test_cases, get_feedback_rounds
)
from dotenv import load_dotenv
load_dotenv(dotenv_path='env.txt')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
sys.path.append('domain_investment')

# Initialize the baseline agent (only has episodic and semantic memory)
agent = CoALABaselineAgent(model_name="gpt-4.1-mini", temperature=0, persist_directory="./baseline_memory_store")

# TESTING:
# Test the baseline agent
response = agent.process_message(
    "I'm looking to rebalance my portfolio. I'm 35 and have moderate risk tolerance.",
    user_id="demo_investor"
)
print("Baseline Response (without procedural memory):")
print(response)
print(f"\nMemory Stats: {agent.get_memory_stats()}")
print("\nNote: Baseline agent has episodic + semantic memory only")
print("Cells 2-5 will add procedural memory capabilities")

/Users/keithbourne/development/notebookenv/genainotebooks/2ndEdition/CHAPTER18/baseline_agent.py:65: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  self.vector_store = Chroma(


Baseline Response (without procedural memory):
Given that you're 35 with a moderate risk tolerance, a balanced portfolio might typically include a mix of equities and fixed income to provide growth potential while managing risk. A common approach is to allocate around 60-70% to stocks and 30-40% to bonds or other fixed-income assets. 

For example:
- 60% equities (diversified across large-cap, mid-cap, international stocks)
- 35% bonds (a mix of government and corporate bonds)
- 5% cash or cash equivalents for liquidity

This allocation aims to capture growth over the long term while providing some stability. Would you like me to help you tailor this further based on your current holdings or specific goals?

Memory Stats: {'total_documents': 9, 'current_user': 'demo_investor', 'current_conversation': 'conv_1758749497.086438'}

Note: Baseline agent has episodic + semantic memory only
Cells 2-5 will add procedural memory capabilities


In [3]:
# Cell 2: Define the structure for procedural memory (NOW GENERIC)
class DomainProcedure(BaseModel):
    """Generic procedure structure for any domain.
    
    Core fields (required for all domains):
    - strategy_pattern: Description of the strategy
    - steps: Ordered list of steps to execute
    - segments: Applicable user segments
    - success_rate: Historical success rate (0.0-1.0)
    - scope: Hierarchy level (global/user/community/task)
    
    Domain-specific data goes in:
    - domain_metrics: Dict for any domain-specific metrics
      (e.g., avg_portfolio_performance for investments,
             quiz_avg for tutoring, etc.)
    """
    # Core fields (domain-agnostic)
    strategy_pattern: str
    steps: List[str]
    segments: List[str] = Field(default_factory=list)
    success_rate: float = 1.0
    usage_count: int = 0
    adaptations: List[Dict] = Field(default_factory=list)
    
    # Segmentation metadata
    scope: str = "global"  
    scope_id: Optional[str] = None
    priority: int = 0
    learned_from_count: int = 1
    
    # Domain-specific metrics stored as flexible dict
    domain_metrics: Dict[str, float] = Field(default_factory=dict)

domain_agent = InvestmentAdvisorAgent()

# TESTING:

# Create a sample procedure for investment domain
sample_procedure = DomainProcedure(
    strategy_pattern="moderate risk portfolio rebalancing",
    steps=[
        "1. Review current allocation and drift from target",
        "2. Assess client's life changes affecting risk tolerance",
        "3. Analyze market conditions and sector rotation opportunities",
        "4. Propose rebalancing with tax-loss harvesting considerations",
        "5. Set up automatic rebalancing schedule"
    ],
    segments=["millennials", "moderate_risk"],  # Generic 'segments' not 'client_segments'
    domain_metrics={
        "avg_portfolio_performance": 8.5,  # Investment-specific metric
        "avg_rebalance_frequency_days": 90  # Another domain metric
    }
)

print("✓ Domain procedure structure created")
print(f"  Strategy: {sample_procedure.strategy_pattern}")
print(f"  Steps: {len(sample_procedure.steps)}")
print(f"  Segments: {', '.join(sample_procedure.segments)}")
print(f"  Domain metrics: {sample_procedure.domain_metrics}")

✓ Domain procedure structure created
  Strategy: moderate risk portfolio rebalancing
  Steps: 5
  Segments: millennials, moderate_risk
  Domain metrics: {'avg_portfolio_performance': 8.5, 'avg_rebalance_frequency_days': 90.0}


In [4]:
# Cell 3: Initialize Procedural Memory System

# Create domain agent (encapsulates all investment-specific logic)
domain_agent = InvestmentAdvisorAgent()

# Create procedural memory with the domain agent
investment_memory = ProceduralMemory(llm=agent.llm, domain_agent=domain_agent)

# TESTING:

print("✓ Procedural memory system initialized")
print(f"  Domain: {domain_agent.__class__.__name__}")

# Verify the system is initialized correctly
stats = investment_memory.get_stats()
print(f"\n📊 Initial state:")
print(f"  Total strategies: {stats['total_strategies']}")
print(f"  By scope: {stats['by_scope']}")
print(f"  Learning history tracking: {len(investment_memory.global_learning_history)} events")

✓ Procedural memory system initialized
  Domain: InvestmentAdvisorAgent

📊 Initial state:
  Total strategies: 0
  By scope: {'global': 0, 'user': 0, 'community': 0, 'task': 0}
  Learning history tracking: 0 events


In [5]:
# Cell 4: Demonstrate Learning from Interactions
# Simulate some successful interactions to learn from
successful_interactions = [
    {
        "query": "I want to rebalance my portfolio",
        "user_id": "user_001",
        "profile": {"age": 35, "risk_tolerance": "moderate"},
        "interaction": {
            "messages": ["User: I want to rebalance", "Assistant: Here's your rebalancing plan..."],
            "success": True,
            "client_satisfaction": 9,
            "returns": 8.5
        }
    },
    {
        "query": "Show me ESG investment options",
        "user_id": "user_002",
        "profile": {"age": 30, "risk_tolerance": "aggressive"},
        "interaction": {
            "messages": ["User: ESG options?", "Assistant: Here are sustainable funds..."],
            "success": True,
            "client_satisfaction": 8,
            "returns": 7.2
        }
    }
]

print("📚 Learning from successful interactions...")
learned_strategies = {}

for data in successful_interactions:
    result = investment_memory.learn_from_interaction(
        query=data["query"],
        interaction_data=data["interaction"],
        user_id=data["user_id"],
        user_profile=data["profile"]
    )
    
    if result.get("learned"):
        for key, value in result.items():
            if "learned" in key:
                learned_strategies[key] = value
                print(f"  ✓ Learned {key}: {value}")

# Check what was learned
stats = investment_memory.get_stats()
print(f"\n📊 After learning:")
print(f"  Total strategies: {stats['total_strategies']}")
print(f"  By scope: {stats['by_scope']}")

📚 Learning from successful interactions...

📊 After learning:
  Total strategies: 8
  By scope: {'global': 2, 'user': 2, 'community': 2, 'task': 2}


In [6]:
# Cell 5a: Demonstrate Hierarchical Retrieval (user → community → task → global)
print("Setting up strategies at each scope level...")
setup_hierarchy_demo(investment_memory)

print("\nDemonstrating retrieval hierarchy:\n" + "=" * 60)

for user_id, query, profile, expected in get_test_cases():
    strategy = investment_memory.get_investment_strategy(query, profile, user_id)
    print(f"\n🔍 User: {user_id}\n   Query: '{query}'\n   Expected: {expected}")
    
    if strategy:
        print(f"   ✓ Retrieved: {strategy['strategy']}")
        print(f"   → Scope: {strategy['scope'].upper()}")
        print(f"   → Source: {strategy['source']}")
        print(f"   → Confidence: {strategy['confidence']:.0%}")
    else:
        print(f"   ✗ No strategy found")

print("\n📊 Hierarchy Summary:")
stats = investment_memory.get_stats()
for scope, count in stats['by_scope'].items():
    print(f"   {scope}: {count} strategies")

Setting up strategies at each scope level...

Demonstrating retrieval hierarchy:

🔍 User: user_001
   Query: 'I want to rebalance'
   Expected: Should retrieve USER scope (user_001 has personalized strategy)
   ✓ Retrieved: user_user_001_preference_0
   → Scope: USER
   → Source: personalized for user_001
   → Confidence: 85%

🔍 User: user_003
   Query: 'Need investment advice'
   Expected: Should retrieve COMMUNITY scope (user_003 in moderate_professionals)
   ✓ Retrieved: moderate_prof_strategy
   → Scope: COMMUNITY
   → Source: learned from moderate_professionals community
   → Confidence: 85%

🔍 User: user_004
   Query: 'Time to rebalance my portfolio'
   Expected: Should retrieve TASK scope (rebalancing task identified)
   ✓ Retrieved: rebalancing_specialist
   → Scope: TASK
   → Source: specialized for rebalancing
   → Confidence: 88%

🔍 User: user_005
   Query: 'General investment question'
   Expected: Should retrieve GLOBAL scope (no specific matches)
   ✓ Retrieved: portfolio

In [7]:
# Cell 5b: Demonstrate Performance Feedback Loop
print("Testing performance feedback and adaptation...")
print("=" * 60)

# Get strategy and show initial state
test_strategy = investment_memory.global_procedures["general_investment"]
print(f"\nInitial state of 'general_investment' strategy:")
print(f"  Success rate: {test_strategy.success_rate:.0%}")
print(f"  Domain metrics: {test_strategy.domain_metrics}")
print(f"  Adaptations: {len(test_strategy.adaptations)}")

print("\n📈 Applying feedback rounds:")
for i, feedback in enumerate(get_feedback_rounds(), 1):
    expected_score = domain_agent.calculate_success_score(feedback)
    old_rate = test_strategy.success_rate
    
    result = investment_memory.update_from_performance(
        strategy="general_investment",
        performance_data=feedback,
        scope="global"
    )
    
    print(f"\nRound {i}: Satisfaction={feedback['client_satisfaction']}, "
          f"Returns={feedback['returns']:+.1f}%")
    print(f"  Success score: {expected_score:.0%}")
    
    if result.get('updated'):
        print(f"  Success rate: {old_rate:.0%} → {result['new_success_rate']:.0%}")
        print(f"  Trend: {result['performance_trend']}")
        if 'avg_portfolio_performance' in test_strategy.domain_metrics:
            print(f"  Avg portfolio: {test_strategy.domain_metrics['avg_portfolio_performance']:.1f}%")

# Show adaptation history
print(f"\n📚 Adaptation History:")
print(f"  Total adaptations: {len(test_strategy.adaptations)}")
if test_strategy.adaptations:
    for i, adaptation in enumerate(test_strategy.adaptations[-2:], 1):
        print(f"\n  Adaptation {i}:")
        print(f"    Time: {adaptation['timestamp'][:19]}")
        print(f"    Old rate: {adaptation['old_rate']:.0%}")
        print(f"    New rate: {adaptation['new_rate']:.0%}")
        print(f"    Success score: {adaptation['success_score']:.0%}")

Testing performance feedback and adaptation...

Initial state of 'general_investment' strategy:
  Success rate: 75%
  Domain metrics: {}
  Adaptations: 0

📈 Applying feedback rounds:

Round 1: Satisfaction=9, Returns=+12.5%
  Success score: 100%
  Success rate: 75% → 80%
  Trend: improving
  Avg portfolio: 1.2%

Round 2: Satisfaction=8, Returns=+8.0%
  Success score: 100%
  Success rate: 80% → 84%
  Trend: improving
  Avg portfolio: 1.9%

Round 3: Satisfaction=4, Returns=-2.0%
  Success score: 0%
  Success rate: 84% → 67%
  Trend: declining
  Avg portfolio: 1.5%

Round 4: Satisfaction=7, Returns=+5.0%
  Success score: 70%
  Success rate: 67% → 68%
  Trend: improving
  Avg portfolio: 1.9%

📚 Adaptation History:
  Total adaptations: 4

  Adaptation 1:
    Time: 2025-09-24T14:32:22
    Old rate: 84%
    New rate: 67%
    Success score: 0%

  Adaptation 2:
    Time: 2025-09-24T14:32:22
    Old rate: 67%
    New rate: 68%
    Success score: 70%


In [8]:
# Cell 5c: Visualize the Procedural Memory Structure

print("Procedural Memory Structure Visualization")
print("=" * 60)

# Show the actual hierarchy
investment_memory.show_strategy_performance()

# Show community membership
print("\n👥 Community Membership Map:")
for user, communities in investment_memory.user_communities.items():
    print(f"  {user}: {', '.join(communities)}")
for community, members in investment_memory.community_members.items():
    if members:
        print(f"  {community} has {len(members)} members: {', '.join(members)}")

# Show discovered segments
print(f"\n🏷️ Discovered Segments: {', '.join(investment_memory.segments_discovered)}")

Procedural Memory Structure Visualization

📊 Strategy Performance by Scope:

🌍 GLOBAL STRATEGIES (Universal Best Practices):
  portfolio_rebalancing          ████████░░ 85.0%
    Used 1x | Segments: moderate_risk, millennials
  esg_investment_selection       ████████░░ 85.0%
  general_investment             ██████░░░░ 67.8%

👤 USER-SPECIFIC STRATEGIES:
  Total users with personalized strategies: 2
  Total personalized procedures: 2

  Example - User user_001:
    • user_user_001_preference_0 (success: 85.0%)

👥 COMMUNITY STRATEGIES:
  moderate_professionals: 1 strategies
    Members: 1 | Avg success: 85.0%
  aggressive_millennials: 1 strategies
    Members: 1 | Avg success: 85.0%

📋 TASK-SPECIFIC STRATEGIES:
  rebalancing: 1 procedures (avg success: 88.0%)
  esg_investing: 1 procedures (avg success: 85.0%)

👥 Community Membership Map:
  user_001: moderate_professionals
  user_002: aggressive_millennials
  user_003: moderate_professionals
  moderate_professionals has 1 members: user_001

In [9]:
# Cell 5d: Test Edge Cases and Fallbacks

print("Testing Edge Cases")
print("=" * 60)

# 1. Empty query
print("\n1. Empty/vague query:")
strategy = investment_memory.get_investment_strategy("", {}, None)
print(f"   Result: {'Strategy found' if strategy else 'No strategy (expected)'}")

# 2. User with no community
print("\n2. User with unassigned community:")
orphan_user = "orphan_user"
strategy = investment_memory.get_investment_strategy(
    "investment advice", 
    {"age": 200, "risk_tolerance": "unknown"},
    orphan_user
)
if strategy:
    print(f"   Fell back to: {strategy['scope']} scope")

# 3. Conflicting scopes - what wins?
print("\n3. Query matching multiple scopes:")
investment_memory.user_procedures["user_001"]["rebalancing_user"] = DomainProcedure(
    strategy_pattern="rebalancing_user",
    steps=["User-specific rebalancing"],
    success_rate=0.95,
    scope="user"
)
strategy = investment_memory.get_investment_strategy(
    "rebalance portfolio",  # Matches both user AND task
    {"age": 35, "risk_tolerance": "moderate"},
    "user_001"
)
print(f"   Winner: {strategy['scope']} scope (user > task in hierarchy)")

Testing Edge Cases

1. Empty/vague query:
   Result: Strategy found

2. User with unassigned community:
   Fell back to: global scope

3. Query matching multiple scopes:
   Winner: user scope (user > task in hierarchy)


In [10]:
# Cell 5e: Crowdsourced Strategy Discovery - Building a Knowledge Moat

print("CROWDSOURCED LEARNING: How User Interactions Build Competitive Advantage")
print("=" * 70)

# Simulate realistic user interactions over time
print("\n📅 PHASE 1: Early Adopters Discover Patterns")
print("-" * 40)

early_adopters = [
    ("early_adopter_001", "ESG tech stocks", {"age": 32, "risk_tolerance": "moderate"},
     {"success": True, "client_satisfaction": 9, "returns": 18.5}),
    ("early_adopter_002", "Tax loss harvesting", {"age": 45, "risk_tolerance": "moderate"},
     {"success": True, "client_satisfaction": 10, "returns": 2.3}),
    ("early_adopter_003", "Defensive rotation", {"age": 58, "risk_tolerance": "conservative"},
     {"success": True, "client_satisfaction": 8, "returns": -1.2}),
]

for user_id, query, profile, performance in early_adopters:
    result = investment_memory.learn_from_interaction(
        query, 
        {"messages": [f"User: {query}", "Assistant: Strategy provided"],
         **performance},
        user_id, 
        profile
    )
    if result.get("learned"):
        print(f"✓ {user_id}: Discovered pattern ({list(result.keys())})")

# Simulate more interactions to build critical mass
print("\n📅 PHASE 2: Pattern Validation Through Multiple Users")
print("-" * 40)

# Simulate 20 more users validating patterns
for i in range(20):
    user_id = f"user_{i:03d}"
    # Users discover and validate existing patterns
    investment_memory.learn_from_interaction(
        ["ESG investing", "Tax optimization", "Risk management"][i % 3],
        {"messages": ["Q", "A"], "success": True, 
         "client_satisfaction": 7 + (i % 3), "returns": 5 + (i % 10)},
        user_id,
        {"age": 30 + i, "risk_tolerance": ["moderate", "aggressive", "conservative"][i % 3]}
    )

# Calculate actual metrics
stats = investment_memory.get_stats()
total_strategies = stats['total_strategies']
users_contributed = len(investment_memory.user_procedures)

# Calculate total adaptations
total_adaptations = 0
total_usage = 0
avg_success_rates = []

for scope_procs in [
    investment_memory.global_procedures.values(),
    *[procs.values() for procs in investment_memory.user_procedures.values()],
    *[procs.values() for procs in investment_memory.community_procedures.values()],
    *[procs.values() for procs in investment_memory.task_procedures.values()]
]:
    for proc in scope_procs:
        total_adaptations += len(proc.adaptations)
        total_usage += proc.usage_count
        avg_success_rates.append(proc.success_rate)

overall_success_rate = sum(avg_success_rates) / len(avg_success_rates) if avg_success_rates else 0

print(f"Validated {total_strategies} unique strategies across {users_contributed} users")

# Test with new users
print("\n📅 PHASE 3: New Users Benefit Immediately")
print("-" * 40)

new_users_before_learning = []
new_users_after_learning = []

# Test BEFORE any learning (baseline)
test_query = "How should I invest in ESG?"
baseline_strategy = investment_memory.get_investment_strategy(
    test_query, {"age": 30, "risk_tolerance": "moderate"}, "new_user_baseline"
)

# Now test AFTER learning
for i in range(3):
    user_id = f"new_user_{i:03d}"
    strategy = investment_memory.get_investment_strategy(
        test_query, {"age": 30, "risk_tolerance": "moderate"}, user_id
    )
    if strategy:
        new_users_after_learning.append({
            "user": user_id,
            "scope": strategy['scope'],
            "confidence": strategy['confidence'],
            "source": strategy['source']
        })

# Show the difference
print(f"New user experience:")
print(f"  Without crowdsourced learning: {baseline_strategy['scope'] if baseline_strategy else 'No strategy'}")
if new_users_after_learning:
    latest = new_users_after_learning[0]
    print(f"  With crowdsourced learning: {latest['scope']} ({latest['confidence']:.0%} confidence)")

print("\n" + "=" * 70)
print("COMPETITIVE ADVANTAGE METRICS (ACTUAL DATA):")
print("=" * 70)

# 1. SCALE METRICS
print("\n1. SCALE ADVANTAGE:")
print(f"   • Strategies discovered: {total_strategies}")
print(f"   • Users contributed: {users_contributed}")
print(f"   • Performance updates: {total_adaptations}")
print(f"   • Total strategy uses: {total_usage}")

# 2. QUALITY METRICS
validation_per_strategy = total_usage / total_strategies if total_strategies > 0 else 0
print("\n2. QUALITY ADVANTAGE:")
print(f"   • Avg validations per strategy: {validation_per_strategy:.1f}")
print(f"   • Overall success rate: {overall_success_rate:.1%}")
print(f"   • Strategies by scope: {stats['by_scope']}")

# 3. PERSONALIZATION METRICS
personalization_coverage = {
    "user_specific": len(investment_memory.user_procedures),
    "community_specific": len(investment_memory.community_procedures),
    "task_specific": len(investment_memory.task_procedures),
    "global_fallback": len(investment_memory.global_procedures)
}
print("\n3. PERSONALIZATION ADVANTAGE:")
for level, count in personalization_coverage.items():
    print(f"   • {level}: {count} strategies")

# 4. GROWTH PROJECTION
days_simulated = 1
strategies_per_day = total_strategies / days_simulated
print("\n4. COMPOUNDING EFFECTS (PROJECTION):")
print(f"   • Day 1: {total_strategies} strategies")
print(f"   • Day 30: ~{int(strategies_per_day * 30)} strategies")
print(f"   • Day 365: ~{int(strategies_per_day * 365)} strategies")
print(f"   • Growth rate: {strategies_per_day:.1f} strategies/day")

# 5. COMPETITIVE MOAT
print("\n5. COMPETITOR BARRIERS:")
competitor_baseline_success = 0.5  # Assumed success without learning
our_success = overall_success_rate
advantage = our_success - competitor_baseline_success

print(f"   • Our success rate: {our_success:.1%}")
print(f"   • Competitor baseline: {competitor_baseline_success:.1%}")
print(f"   • Our advantage: +{advantage:.1%}")
print(f"   • Strategies they lack: {total_strategies}")
print(f"   • Validations they lack: {total_usage}")
print(f"   • Time to catch up: {total_strategies / strategies_per_day:.0f} days at our rate")

# Show a specific example of competitive advantage
if investment_memory.global_procedures:
    best_strategy = max(investment_memory.global_procedures.values(), 
                       key=lambda x: x.success_rate)
    print(f"\n📊 Example - Our best strategy:")
    print(f"   • Pattern: {best_strategy.strategy_pattern}")
    print(f"   • Success rate: {best_strategy.success_rate:.1%}")
    print(f"   • Validated: {best_strategy.usage_count} times")
    print(f"   • Learned from: {best_strategy.learned_from_count} users")
    if best_strategy.domain_metrics:
        print(f"   • Performance: {best_strategy.domain_metrics}")

print("\n🚀 Every user interaction compounds our competitive advantage")

CROWDSOURCED LEARNING: How User Interactions Build Competitive Advantage

📅 PHASE 1: Early Adopters Discover Patterns
----------------------------------------

📅 PHASE 2: Pattern Validation Through Multiple Users
----------------------------------------
Validated 38 unique strategies across 23 users

📅 PHASE 3: New Users Benefit Immediately
----------------------------------------
New user experience:
  Without crowdsourced learning: task
  With crowdsourced learning: task (85% confidence)

COMPETITIVE ADVANTAGE METRICS (ACTUAL DATA):

1. SCALE ADVANTAGE:
   • Strategies discovered: 38
   • Users contributed: 23
   • Performance updates: 4
   • Total strategy uses: 7

2. QUALITY ADVANTAGE:
   • Avg validations per strategy: 0.2
   • Overall success rate: 84.9%
   • Strategies by scope: {'global': 5, 'user': 24, 'community': 2, 'task': 7}

3. PERSONALIZATION ADVANTAGE:
   • user_specific: 23 strategies
   • community_specific: 2 strategies
   • task_specific: 4 strategies
   • global_fa

In [11]:
# # Run this if you want to rerun from a clean slate
# # This clears ALL generated data and memory stores

# import shutil
# import os

# print("🧹 Starting complete cleanup...")

# # 1. Clear the domain memory store (vector database)
# if os.path.exists("./domain_investment/domain_memory_store"):
#     shutil.rmtree("./domain_investment/domain_memory_store")
#     print("  ✓ Cleared domain memory store")

# # 2. Clear the old memory stores from testing (if they exist)
# test_dirs = [
#     "./baseline_memory_store",  # From Cell 1
#     "./full_memory_store",       # Old name from earlier versions
#     "./investment_memory_store"  # Another old name
# ]
# for dir_path in test_dirs:
#     if os.path.exists(dir_path):
#         shutil.rmtree(dir_path)
#         print(f"  ✓ Cleared {dir_path}")

# # 3. Clear generated data (optional - comment out if you want to keep the data)
# if os.path.exists("./domain_investment/investment_advisor_data"):
#     # Only clear the generated files, not the directory itself
#     data_files = [
#         "./domain_investment/investment_advisor_data/conversations.jsonl",
#         "./domain_investment/investment_advisor_data/extracted_patterns.json",
#         "./domain_investment/investment_advisor_data/test_scenarios.json",
#         "./domain_investment/investment_advisor_data/statistics.json"
#     ]
#     for file_path in data_files:
#         if os.path.exists(file_path):
#             os.remove(file_path)
#     print("  ✓ Cleared generated data files")

# # 4. Clear any Chroma persistence files in the root directory
# chroma_files = [
#     "chroma.sqlite3",
#     ".chroma"
# ]
# for file_path in chroma_files:
#     if os.path.exists(file_path):
#         if os.path.isfile(file_path):
#             os.remove(file_path)
#         else:
#             shutil.rmtree(file_path)
#         print(f"  ✓ Cleared {file_path}")

# print("\n✅ Cleanup complete!")
# print("   You can now restart from Cell 6 with a completely fresh system")
# print("\n   Note: The code files (.py) are preserved - only runtime data was cleared")